# LLM-as-Judge

LangSmith 에서 제공되는 Off-the-shelf Evaluators 를 활용해 보겠습니다.

Off-the-shelf Evaluators 는 사전에 정의된 프롬프트 기반의 LLM 평가자를 의미합니다.

쉽게 사용할 수 있는 이점이 있지만, 더 확장된 기능을 사용하기 위해서는 직접 평가자를 정의해야 합니다.

기본적으로 다음의 3가지 정보를 LLM Evaluator 에 전달하여 평가를 진행합니다.

- `input`: 질문. 보통 데이터셋의 Question 이 사용됩니다.
- `prediction`: LLM 이 생성한 답변. 보통 모델의 답변이 사용됩니다.
- `reference`: 정답 답변, Context 등 변칙적으로 활용이 가능.

**참고**
- https://docs.smith.langchain.com/evaluation/faq/evaluator-implementations

In [ ]:
# 설치
# !pip install -qU langsmith langchain-teddynote

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH16-Evaluations")

## RAG 성능 테스트를 위한 함수 정의

테스트에 활용할 RAG 시스템을 생성하겠습니다.

In [ ]:
from myrag import PDFRAG
from langchain_openai import ChatOpenAI

# PDFRAG 객체 생성
rag = PDFRAG(
    "data/SPRI_AI_Brief_2023년12월호_F.pdf",
    ChatOpenAI(model="gpt-4o-mini", temperature=0),
)

# 검색기(retriever) 생성
retriever = rag.create_retriever()

# 체인(chain) 생성
chain = rag.create_chain(retriever)

# 질문에 대한 답변 생성
chain.invoke("삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?")

`ask_question` 이라는 이름으로 함수를 생성합니다. 입력으로는 `inputs` 라는 딕셔너리를 받고, 출력으로는 `answer` 라는 딕셔너리를 반환합니다.

In [ ]:
# 질문에 대한 답변하는 함수를 생성
def ask_question(inputs: dict):
    return {"answer": chain.invoke(inputs["question"])}

In [ ]:
# 사용자 질문 예시
llm_answer = ask_question(
    {"question": "삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?"}
)
llm_answer

evaluator prompt 출력을 위한 함수를 정의합니다.

In [ ]:
# evaluator prompt 출력을 위한 함수
def print_evaluator_prompt(evaluator):
    return evaluator.evaluator.prompt.pretty_print()

## Question-Answer Evaluator

가장 기본 기능을 가진 Evaluator 입니다. 질문(Query) 와 답변(Answer) 을 평가합니다.

사용자 입력은 `input` 으로 LLM 이 생성한 답변은 `prediction` 으로 정답 답변은 `reference` 로 정의됩니다.

(하지만, Prompt 변수는 `query`, `result`, `answer` 로 정의됩니다.)

- `query`: 질문
- `result`: LLM 답변
- `answer`: 정답 답변

In [ ]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator

# qa 평가자 생성
qa_evalulator = LangChainStringEvaluator("qa")

# 프롬프트 출력
print_evaluator_prompt(qa_evalulator)

평가를 진행하고, 출력된 URL 로 이동하여 결과를 확인합니다.

In [ ]:
dataset_name = "RAG_EVAL_DATASET"

# 평가 실행
experiment_results = evaluate(
    ask_question,
    data=dataset_name,
    evaluators=[qa_evalulator],
    experiment_prefix="RAG_EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "QA Evaluator 를 활용한 평가",
    },
)

![](./assets/output-01.png)

## Context 에 기반한 답변 Evaluator

- `LangChainStringEvaluator("context_qa")`: LLM 체인에 정확성을 판단하는 데 참조 "context" 를 사용하도록 지시합니다.
- `LangChainStringEvaluator("cot_qa")`: `"cot_qa"` 는 `"context_qa"` 평가자와 유사하지만, 최종 판결을 결정하기 전에 LLM 의 '추론'을 사용하도록 지시한다는 점에서 차이가 있습니다.

**참고**

먼저, Context 를 반환하는 함수를 정의해야 합니다: `context_answer_rag_answer`

그 다음, `LangChainStringEvaluator` 를 생성합니다. 생성시 `prepare_data` 를 통해 위에서 정의한 함수의 반환 값을 적절하게 매핑합니다.

**세부사항**

- `run`: LLM 이 생성한 결과 (`context`, `answer`, `input`)
- `example`: 데이터셋에 정의된 데이터입니다. (`question` 과 `answer`)

`LangChainStringEvaluator` 이 평가를 수행하기 위하여 다음의 3가지 정보가 필요합니다.

- `prediction`: LLM 이 생성한 답변
- `reference`: 데이터셋에 정의된 답변
- `input`: 데이터셋에 정의된 질문

하지만, `LangChainStringEvaluator("context_qa")` 는 `reference` 를 Context 로 사용하기 때문에 다음과 같이 정의합니다.

(참고) 아래는 `context_qa` 평가자를 활용하기 위하여 `context`, `answer`, `question` 을 반환하는 함수를 정의하였습니다.

In [ ]:
# Context 를 반환하는 RAG 결과 반환 함수
def context_answer_rag_answer(inputs: dict):
    context = retriever.invoke(inputs["question"])
    return {
        "context": "\n".join([doc.page_content for doc in context]),
        "answer": chain.invoke(inputs["question"]),
        "query": inputs["question"],
    }

In [ ]:
# 함수 실행
context_answer_rag_answer(
    {"question": "삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?"}
)

In [ ]:
# cot_qa 평가자 생성
cot_qa_evaluator = LangChainStringEvaluator(
    "cot_qa",
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],  # LLM 이 생성한 답변
        "reference": run.outputs["context"],  # Context
        "input": example.inputs["question"],  # 데이터셋의 질문
    },
)

# context_qa 평가자 생성
context_qa_evaluator = LangChainStringEvaluator(
    "context_qa",
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],  # LLM 이 생성한 답변
        "reference": run.outputs["context"],  # Context
        "input": example.inputs["question"],  # 데이터셋의 질문
    },
)

# evaluator prompt 출력
print_evaluator_prompt(context_qa_evaluator)

평가를 진행하고, 출력된 URL 로 이동하여 결과를 확인합니다.

In [ ]:
# 데이터셋 이름 설정
dataset_name = "RAG_EVAL_DATASET"

# 평가 실행
evaluate(
    context_answer_rag_answer,
    data=dataset_name,
    evaluators=[cot_qa_evaluator, context_qa_evaluator],
    experiment_prefix="RAG_EVAL",
    metadata={
        "variant": "COT_QA & Context_QA Evaluator 를 활용한 평가",
    },
)

![](./assets/output-02.png)

평가 결과 `Ground Truth` 와 맞지 않은 답변을 생성해도 주어진 `Context` 가 맞다면 **CORRECT** 로 평가됩니다.

## Criteria

기준값 참조 레이블(정답 답변)이 없거나 얻기 힘든 경우 `"criteria"` 또는 `"score"` 평가자를 사용하여 사용자 지정 기준 집합에 대해 실행을 평가할 수 있습니다. 

이는 모델의 답변에 대한 **높은 수준의 의미론적 측면을 모니터링** 하려는 경우에 유용합니다.

LangChainStringEvaluator("criteria", config={ "criteria": `아래 중 하나의 criterion` })

| 기준 | 설명 |
|------|------|
| `conciseness` | 답변이 간결하고 간단한지 평가 |
| `relevance` | 답변이 질문과 관련이 있는지 평가 |
| `correctness` | 답변이 옳은지 평가 |
| `coherence` | 답변이 일관성이 있는지 평가 |
| `harmfulness` | 답변이 해롭거나 유해한지 평가 |
| `maliciousness` | 답변이 악의적이거나 악화시키는지 평가 |
| `helpfulness` | 답변이 도움이 되는지 평가 |
| `controversiality` | 답변이 논란이 되는지 평가 |
| `misogyny` | 답변이 여성을 비하하는지 평가 |
| `criminality` | 답변이 범죄를 촉진하는지 평가 |


In [ ]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator

# 평가자 설정
criteria_evaluator = [
    LangChainStringEvaluator("criteria", config={"criteria": "conciseness"}),
    LangChainStringEvaluator("criteria", config={"criteria": "misogyny"}),
    LangChainStringEvaluator("criteria", config={"criteria": "criminality"}),
]

# 데이터셋 이름 설정
dataset_name = "RAG_EVAL_DATASET"

# 평가 실행
experiment_results = evaluate(
    ask_question,
    data=dataset_name,
    evaluators=criteria_evaluator,
    experiment_prefix="CRITERIA-EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "criteria 를 활용한 평가",
    },
)

![](./assets/output-03.png)

## 정답이 존재하는 경우 Evaluator 활용(labeled_criteria)

정답이 존재하는 경우, LLM 이 생성한 답변과 정답 답변을 비교하여 평가가 가능합니다.

아래의 예시처럼 `reference` 에는 정답 답변을, `prediction` 에는 LLM 이 생성한 답변을 전달합니다.

이 처럼 별도의 설정은 `prepare_data` 를 통해 정의합니다.

또한, 답변 평가에 활용되는 LLM 은 `config` 의 `llm` 을 통해 정의합니다.

In [ ]:
from langsmith.evaluation import LangChainStringEvaluator
from langchain_openai import ChatOpenAI

# labeled_criteria 평가자 생성
labeled_criteria_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm": ChatOpenAI(temperature=0.0, model="gpt-4o-mini"),
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],
        "reference": example.outputs["answer"],  # 정답 답변
        "input": example.inputs["question"],
    },
)

# evaluator prompt 출력
print_evaluator_prompt(labeled_criteria_evaluator)

아래는 `relevance` 를 평가하는 예시입니다.

이번에는 `prepare_data` 를 통해 `reference` 를 `context` 로 전달합니다.

In [ ]:
from langchain_openai import ChatOpenAI

relevance_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": "relevance",
        "llm": ChatOpenAI(temperature=0.0, model="gpt-4o-mini"),
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],
        "reference": run.outputs["context"],  # Context 를 전달
        "input": example.inputs["question"],
    },
)

print_evaluator_prompt(relevance_evaluator)

평가를 진행하고, 출력된 URL 로 이동하여 결과를 확인합니다.

In [ ]:
from langsmith.evaluation import evaluate

# 데이터셋 이름 설정
dataset_name = "RAG_EVAL_DATASET"

# 평가 실행
experiment_results = evaluate(
    context_answer_rag_answer,
    data=dataset_name,
    evaluators=[labeled_criteria_evaluator, relevance_evaluator],
    experiment_prefix="LABELED-EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "labeled_criteria evaluator 활용한 평가",
    },
)

![](./assets/output-04.png)

## 사용자 정의 점수 Evaluator(labeled_score_string)

아래는 점수를 반환하는 평가자 생성 예시입니다. `normalize_by` 를 통해 점수를 정규화할 수 있습니다. 변환된 점수는 (0 ~ 1) 사이의 값으로 정규화됩니다.

아래의 `accuracy` 는 사용자가 임의로 정의한 기준입니다. 적합한 Prompt 를 정의하여 사용할 수 있습니다.

In [ ]:
from langsmith.evaluation import LangChainStringEvaluator

# 점수를 반환하는 평가자 생성
labeled_score_evaluator = LangChainStringEvaluator(
    "labeled_score_string",
    config={
        "criteria": {
            "accuracy": "How accurate is this prediction compared to the reference on a scale of 1-10?"
        },
        "normalize_by": 10,
        "llm": ChatOpenAI(temperature=0.0, model="gpt-4o-mini"),
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    },
)

print_evaluator_prompt(labeled_score_evaluator)

평가를 진행하고, 출력된 URL 로 이동하여 결과를 확인합니다.

In [ ]:
from langsmith.evaluation import evaluate

# 평가 실행
experiment_results = evaluate(
    ask_question,
    data=dataset_name,
    evaluators=[labeled_score_evaluator],
    experiment_prefix="LABELED-SCORE-EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "labeled_score 활용한 평가",
    },
)

![](./assets/output-05.png)